**Approach 1.2:**

**Training:**

-> For every selected frame of video, extract features and give label(label generated by converting manually marked safe duration into frame intervals)

-> Use classification to predict 0/1 for each frame

**Testing :**

For every frame, extract features and use classification to predict label

**Features to be extracted from individual frames:**

1. Number of bounding boxes in every region (total region:6)
2. Total Area covered by all bounding boxes in every region
3. Minimum distance of bounding box from every region

In addition, for this approach, direction detection arrays are added to minimize features to necessary ones. if vehicles are in opposite direction, they will be ignored in this approach for generating dataframe.

In [48]:
#importing necessary libraries

from __future__ import print_function
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
%matplotlib inline  
from imageai.Detection import VideoObjectDetection
import os
import sys
from random import randint
from math import ceil, sqrt
import natsort
import pandas as pd
import random
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import GridSearchCV
import sys
import cv2
from random import randint


In [49]:
# constant path variables

name = 'yagnesh'

if name == 'siddhi':
    path_train_arrays = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/arrays_train_v2'
    path_test_arrays = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/arrays_test_v2'
    path_train_labels = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/labels_train.csv'
    path_test_labels = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/labels_test.csv'
    path_train_videos = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/videos_train'
    path_test_videos = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/videos_test'


elif name == 'yagnesh':
    path_train_arrays = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/arrays_train_v2'   # arrays of training set generated using RetinaNet
    path_test_arrays = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/arrays_test_v2'    # arrays of test set generated using RetinaNet
    path_direction_train_arrays = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/directions_train_v2'   # diirection arrays of training set generated using RetinaNet
    path_direction_test_arrays = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/directions_test_v2'# direction arrays of training set generated using RetinaNet
    path_train_labels = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_train.csv' # training labels file
    path_test_labels = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_test.csv'  # testing labels file

In [50]:
def find_region(x,y):

    '''
    Returns the region in which the particular point lies

    Parameters:
    x(int) : x coordinate of the point
    y(int) : y coordinate of the point

    Returns:
    int : region no (1/2/3/4/5/6)
    '''


    video_height = 1080
    video_width = 1920

    if(x<640 and y>=540):
        return 1 #Bottom left region
    if(x<=1280 and y>=540):
        return 2 #Bottom center region
    if(x>1280 and y>=540):
        return 3 #Bottom right region
    if(x>1280 and y<540):
        return 4 #Top right region
    if(x<=1280 and y<540):
        return 5 #Top center region
    if(x<640 and y<540):
        return 6 #Top left region

def extract_features_from_frame(bounding_boxes, directions_for_boxes, frame_no):
        
    '''
    Extract features from a particular frame

    Parameters:
    bounding_boxes(array) : detection array of the video
    direction_for_boxes(array) : detection of direction for all bounding boxes 
    frame_no(int) : frame no for which features are to be extracted

    Returns:
    list : containing 18 features of the frame
    '''    

    no_of_boxes = len(bounding_boxes[frame_no])  # total number of bounding boxes
    boxes_frame = np.array(bounding_boxes[frame_no]) # bounding boxes for each frame 
    direction_frame = np.array(directions_for_boxes[frame_no]) # directions for every bounding boxes

    # checking if no of bounding boxes and direction array length matches or not 
    if len(direction_frame) != no_of_boxes:
        raise Exception('mismatch in number of boxes '+ str(len(direction_frame))+' '+str(no_of_boxes))

    # matching indices of bounding boxes with required directions array
    indices_required_boxes = np.where(direction_frame>=0)[0] # getting direction indices which is having value 0 or 1 (steady or positive direction vehicle)
    boxes_required = boxes_frame[indices_required_boxes] # assigning indices of interested direction values (0 and 1 value of directions)

    # feature - number of vehicles in each region
    r1 = r2 = r3 = r4 = r5 = r6 = 0
    
    # feature - total area of vehicles in each region
    r1_area = r2_area = r3_area = r4_area = r5_area = r6_area = 0

    # distance calculation for every region
    bottom_center_point_x = 1920/2    # width/2
    bottom_center_point_y = 1080      # height
    r1_min_distance = r2_min_distance = r3_min_distance = r4_min_distance = r5_min_distance = r6_min_distance = 0

    for j in range(len(boxes_required)):

        # center x coordinate of bounding box cx
        cx = (boxes_required[j][0] + boxes_required[j][2])/2

        # center y coordinate of bounding box cy
        cy = (boxes_required[j][1] + boxes_required[j][3])/2

        # area of bounding box
        bounding_box_area = ((boxes_required[j][3]-boxes_required[j][1])*
                        (boxes_required[j][2]-boxes_required[j][0]))

        # distance calculation for every bounding box
        distance = sqrt((cx - bottom_center_point_x)**2 + (cy - bottom_center_point_y)**2)

        # finding region according to cx,cy and finding total area and minimum distance for each region
        if(find_region(cx,cy)==1):
            r1 = r1 + 1
            r1_area = r1_area + bounding_box_area
            if(r1_min_distance==0 or distance < r1_min_distance):
                r1_min_distance = distance
        elif(find_region(cx,cy)==2):
            r2 = r2 + 1
            r2_area = r2_area + bounding_box_area
            if(r2_min_distance==0 or distance < r2_min_distance):
                r2_min_distance = distance
        elif(find_region(cx,cy)==3):
            r3 = r3 + 1
            r3_area = r3_area + bounding_box_area
            if(r3_min_distance==0 or distance < r3_min_distance):
                r3_min_distance = distance
        elif(find_region(cx,cy)==4):
            r4 = r4 + 1
            r4_area = r4_area + bounding_box_area
            if(r4_min_distance==0 or distance < r4_min_distance):
                r4_min_distance = distance
        elif(find_region(cx,cy)==5):
            r5 = r5 + 1
            r5_area = r5_area + bounding_box_area
            if(r5_min_distance==0 or distance < r5_min_distance):
                r5_min_distance = distance
        elif(find_region(cx,cy)==6):
            r6 = r6 + 1
            r6_area = r6_area + bounding_box_area
            if(r6_min_distance==0 or distance < r6_min_distance):
                r6_min_distance = distance
                
    features_frame = [r1,r1_area,r1_min_distance,r2,r2_area,r2_min_distance,r3,r3_area,r3_min_distance,r4,r4_area,r4_min_distance,r5,r5_area,r5_min_distance,r6,r6_area,r6_min_distance]  # final frame with 18 features
        
    return features_frame

In [51]:
def get_labels_from_video(findex, no_frames, labels_csv):
    
    '''
    Get labels for a particular video 

    Parameters:
    findex(int) : video number in labels_csv for which labels are to be generated(starting from 0)
    no_frames(int) : no of frames in the given video
    labels_csv : the manually marked labels

    Returns:
    list : list with len = no of frames and the value at each index represents safe/unsafe at that frame_no (frame_no starting at 0)
    int : -1 if there is no safe duration in video, 1 otherwise
    '''

    labels = [0]*no_frames
    if(np.isnan(labels_csv[findex][0])):
        return labels,-1 # there is no safe duration in the given video so all labels marked 0
    else:
        j = 0
        while(j<len(labels_csv[findex]) and not(np.isnan(labels_csv[findex][j]))):
            # mapping safe_start and safe_end in seconds to frame no
            # frame number starts with 0
            safe_start = max(int(labels_csv[findex][j]*30)-1,0) 
            safe_end = min(int(labels_csv[findex][j+1]*30)-1, no_frames-1)
            labels[safe_start:safe_end+1] = [1]*(safe_end-safe_start+1) # marking the value b/w safe_start and safe_end with 1
            j = j+2
    if len(labels) > no_frames: # len of labels cannot be greater than no_frames in video
        raise Exception('Check the labels assigned in CSV file!')
    return labels,1


In [52]:
# alternate generate_datafrmae function with frames selected from unsafe videos
def generate_dataframe(arrays_folder, direction_arrays_folder, findex, labels_csv):

    '''
    Generate dataframes for model training

    Parameters:
    arrays_folder(path) : folder containing detection arrays
    direction_arrays_folder(path) : folder containing directions array
    findex(int) : array number to start with from the given folder (starting from 0)
    labels_csv : the manually marked labels

    Returns:
    features_dataframe : dataframe containg 18 features per frame
    labels_dataframe : dataframe containing 1 label per frame
    '''
    
    #getting all paths of all arrays in a list in proper order 
    arrays = glob.glob(arrays_folder+'/array*.npy')
    arrays = natsort.natsorted(arrays)

    # getting all paths of all direction arrays in a list in proper order
    direction_arrays = glob.glob(direction_arrays_folder+'/directions*.npy')
    direction_arrays = natsort.natsorted(direction_arrays)

    cols_f = ['region1','region1_area','region1_min_distance','region2','region2_area','region2_min_distance','region3','region3_area','region3_min_distance','region4','region4_area','region4_min_distance','region5','region5_area','region5_min_distance','region6','region6_area','region6_min_distance'] #18 features per frame
    cols_l = ['safe/unsafe'] #1 label per frame
    features_dataframe = pd.DataFrame()
    labels_dataframe = pd.DataFrame()

    for fname,directions_path in zip(arrays, direction_arrays):
        
        print("processing ",fname,":")
        bounding_boxes = np.load(fname, allow_pickle=True) #loading the numpy array containing all detected vehicles
        directions_for_boxes = np.load(directions_path, allow_pickle=True) # loading the arrays containing detected directions for each bounding box
        no_frames = bounding_boxes.shape[0]
        print("no of frames: ", no_frames)
        labels, flag = get_labels_from_video(findex, no_frames, labels_csv)
        print('flag', flag)

        labels_0 = [i for i, value in enumerate(labels) if value == 0 and i>6]
        # shuffling unsafe frames
        random.shuffle(labels_0)
        # selecting 50 frames from unsafe videos instead of rejecting the whole video
        if(len(labels_0) >= 50): 
            labels_0 = labels_0[:50]

        if(flag == -1):
            print('no safe time')
            labels_1 = []
        else:
            labels_1 = [i for i, value in enumerate(labels) if value == 1 and i>6]
            # shuffling safe frames
            random.shuffle(labels_1)
            ## selecting no of safe frames = no of unsafe frames from each video ##
            # label 0 frames will be more than label 1 frame, 
            # so we will try to select same number of safe and unsafe frames
            if(len(labels_0) >= len(labels_1)):
                labels_0 = labels_0[:len(labels_1)]

        features = []

        # extracting features from labels_0 and labels_1 frames
        for frame_no in labels_0:
            features_frame = extract_features_from_frame(bounding_boxes, directions_for_boxes, frame_no)
            features.append(features_frame)
        for frame_no in labels_1:
            features_frame = extract_features_from_frame(bounding_boxes, directions_for_boxes, frame_no)
            features.append(features_frame)
        
        # generating and returning features and labels dataframes
        no_frames_in_features = len(features)
        print("no_frames_in_features",no_frames_in_features)
        labels = [0]*len(labels_0) + [1]*len(labels_1)
        print("no_of_frames_in_labels", len(labels))
        findex = findex + 1
        df1 = pd.DataFrame(features, columns=cols_f)
        features_dataframe = features_dataframe.append(df1,ignore_index=True)
        print("df1.size",df1.shape)
        df2 = pd.DataFrame(labels, columns=cols_l)
        labels_dataframe = labels_dataframe.append(df2,ignore_index=True)
        print("df2.size",df2.shape)
        
    return features_dataframe, labels_dataframe

In [53]:
from numpy import genfromtxt

labels_train = genfromtxt(path_train_labels, delimiter =',')
labels_test = genfromtxt(path_test_labels, delimiter =',')
print(labels_train.shape, labels_test.shape)

(60, 4) (16, 6)


## Generating Dataframe

In [54]:
features_df_train, labels_df_train = generate_dataframe(path_train_arrays, path_direction_train_arrays,0,labels_train)
# features_df_train.to_pickle('features_df_train.pkl')
# labels_df_train.to_pickle('labels_df_train.pkl')

features_df_test, labels_df_test = generate_dataframe(path_test_arrays, path_direction_test_arrays, 0,labels_test)
# features_df_test.to_pickle('features_df_test.pkl')
# labels_df_test.to_pickle('labels_df_test.pkl')
print(features_df_train.shape)

processing  /home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/arrays_train_v2/array1.npy :
no of frames:  210
flag -1
no safe time
no_frames_in_features 50
no_of_frames_in_labels 50
df1.size (50, 18)
df2.size (50, 1)
processing  /home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/arrays_train_v2/array2.npy :
no of frames:  150
flag 1
no_frames_in_features 143
no_of_frames_in_labels 143
df1.size (143, 18)
df2.size (143, 1)
processing  /home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/arrays_train_v2/array3.npy :
no of frames:  180
flag 1
no_frames_in_features 111
no_of_frames_in_labels 111
df1.size (111, 18)
df2.size (111, 1)
processing  /home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/arrays_train_v2/array4.npy :
no of frames:  150
flag -1
no safe time
no_frames_in_features 50
no_of_frames_in_labels 50
df1.size (50, 18)
df2.size (50, 1)
processing  /home/yagnesh/Study/Machine 

## Feature scaling
Strategies that can be used: 
### 1. MinMaxScaler
### 2. StandardScaler

In [55]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# StandardScaler
feature_scaler = StandardScaler()

# MinMaxScaler
# feature_scaler = MinMaxScaler()

# storing scaled features
features_train_scaled = pd.DataFrame(feature_scaler.fit_transform(features_df_train))
features_train_scaled.columns = ['region1','region1_area','region1_min_distance','region2','region2_area','region2_min_distance','region3','region3_area','region3_min_distance','region4','region4_area','region4_min_distance','region5','region5_area','region5_min_distance','region6','region6_area','region6_min_distance']
features_test_scaled = pd.DataFrame(feature_scaler.transform(features_df_test))
features_test_scaled.columns = ['region1','region1_area','region1_min_distance','region2','region2_area','region2_min_distance','region3','region3_area','region3_min_distance','region4','region4_area','region4_min_distance','region5','region5_area','region5_min_distance','region6','region6_area','region6_min_distance']

## Model Training
### Classification model: Support Vector Machine with rbf kernel

In [56]:
# scaled dataset
model = SVC(random_state=0)
model.fit(features_train_scaled, labels_df_train) 
from sklearn.metrics import accuracy_score
 
# print prediction results 
print("for train data:")
predictions = model.predict(features_train_scaled) 
print(classification_report(labels_df_train, predictions)) 
print("accuracy score", accuracy_score(labels_df_train, predictions))

print("\nfor test data:")
predictions = model.predict(features_test_scaled) 
print(classification_report(labels_df_test, predictions)) 
print("accuracy score",accuracy_score(labels_df_test, predictions))


for train data:
              precision    recall  f1-score   support

           0       0.82      0.74      0.78      2925
           1       0.79      0.85      0.82      3252

    accuracy                           0.80      6177
   macro avg       0.80      0.80      0.80      6177
weighted avg       0.80      0.80      0.80      6177

accuracy score 0.799740974583131

for test data:
              precision    recall  f1-score   support

           0       0.61      0.68      0.64       722
           1       0.80      0.74      0.77      1231

    accuracy                           0.72      1953
   macro avg       0.70      0.71      0.70      1953
weighted avg       0.73      0.72      0.72      1953

accuracy score 0.7183819764464926
